In [2]:
from utils import get_personalized_data
from ml import myModel, find_best_SVC
import numpy as np
import os
import joblib

In [5]:
# MODELS = ["KNN", "RF", "SVC", "MLP"]
MODELS = ["KNN"]


def run_personalized_exp(view_idx=1, best_params=False):
    personalized_data = get_personalized_data(view_idx=view_idx)
    RESULT = {
        "KNN": [],
        "RF": [],
        "MLP": [],
        "SVC": []
    }
    best_acc = dict(KNN=0.0, RF=0.0, MLP=0.0, SVC=0.0)
    save_folder = "./output/personalzation/"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    for data_train, data_test in personalized_data:
        for model_name in MODELS:
            my_model = myModel(model_name)
            if model_name == "SVC" and best_params:
                best_params = find_best_SVC(data_train[0], data_train[1])
                my_model.model.set_params(**best_params)
            my_model.train(data_train)
            result = my_model.test(data_test)
            RESULT[model_name].append(result)
            acc = result['accuracy_score']
            if acc > best_acc[model_name]:
                best_acc[model_name] = acc
                joblib.dump(my_model.model, os.path.join(save_folder,, f"{model_name}.pkl"))
                
    return RESULT

def show_results(results):
    for model_name in MODELS:
        acc = np.array([item["accuracy_score"] for item in results[model_name]])
        f1 = np.array([item["f1_score"] for item in results[model_name]])
        recall = np.array([item["recall_score"] for item in results[model_name]])
        
        print(f"{model_name}")
        print(f"acc: {acc.mean() * 100: .2f}  {acc.std() * 2 * 100: .2f}")
        print(f"f1: {f1.mean() * 100: .2f}  {f1.std() * 2 * 100: .2f}")
        print(f"recall: {recall.mean() * 100: .2f}  {recall.std() * 2 * 100: .2f}")

In [7]:
results = run_personalized_exp()
show_results(results)

KNN
acc:  52.72   21.39
f1:  46.33   18.36
recall:  50.55   18.32
